## Dependencies

install condas & nb_conda

In [214]:
import pandas as pd
import numpy as np
import jsonschema
import json
from functools import reduce

In [196]:
covidom_struc = pd.read_csv('../doc/covidom_struc.csv', sep=';')

In [197]:
covidom_struc.columns

Index(['table', 'data_type', 'source_table', 'category', 'complexity',
       'primary_keys', 'unique_fields', 'foreign_keys', 'comments'],
      dtype='object')

In [198]:
covidom_struc.head(7)

,table,data_type,source_table,category,complexity,primary_keys,unique_fields,foreign_keys,comments
0,patient,patient,?,patients et suivis,élémentaire,"[""patient""]","[""patient""]",[],x
1,clinic,établissement de santé?,Clinic,tables de référence,élémentaire,"[""Id""]","[""Id""]",[],"1 seule ligne : ""1|Région Ile de France|1"""
2,doctor_list,médecin,Doctor,tables de référence,élémentaire,"[""#Doctor""]","[""#Doctor""]",[],Medecin fait référence à la même table
3,clinic_followup_reason,raison de suivi,?,patients et suivis,complexe,"[""#Clinic"", ""#Follow up reason""]?","[""#Clinic"", ""#Follow up reason""]?","[{""#Clinic"": {""Id"": ""clinic""}}]","PK is [""#Clinic"", ""#Follow up reason""] or [""#F..."
4,patient_note,données patients hors authentification,?,patients et suivis,complexe,"[""UId""]","[""UId""]","[{""PatientUId"": {""#Patient"": ""patient""}}]",référence à Author_id qui n'est pas présente
5,patient_stay,séjour_de_patient,PatientStay,patients et suivis,complexe,"[""#PatientStay_UId""]","[""#PatientStay_UId""]","[{""PatientStay_PatientUId"": {""#Patient"": ""pati...","manque pk pour {""PatientStay_UserProfileUId"", ..."
6,doctor,médecins dans leurs établissements,Doctor_Clinic,tables de référence,complexe,NaN,NaN,"[{""#Doctor"": {""#Doctor"": ""doctor_list""}}, {""#C...",relation doctor - clinic doctor 1 - n ?


In [199]:
covidom_struc.foreign_keys[5]

'[{"PatientStay_PatientUId": {"#Patient": "patient"}}, {"PatientStay_ClinicId": {"#Clinic": "clinic_followup_reason"}}, {"PatientStay_DoctorId": {"#Doctor": "doctor_list"}}, {"PatientStay_FollowUpReasonId": {"#Follow up reason": "clinic_followup_reason"}}, {"PatientStay_EndReasonId": {"Id": "end_reason"}}, {"PatientStay_MedicalDepartmentId": {"#Medical dept": "medical_department_list" }}]'

In [200]:
json.loads(covidom_struc.foreign_keys[5])

[{'PatientStay_PatientUId': {'#Patient': 'patient'}},
 {'PatientStay_ClinicId': {'#Clinic': 'clinic_followup_reason'}},
 {'PatientStay_DoctorId': {'#Doctor': 'doctor_list'}},
 {'PatientStay_FollowUpReasonId': {'#Follow up reason': 'clinic_followup_reason'}},
 {'PatientStay_EndReasonId': {'Id': 'end_reason'}},
 {'PatientStay_MedicalDepartmentId': {'#Medical dept': 'medical_department_list'}}]

In [201]:
covidom_struc['json_fk'] = covidom_struc.foreign_keys.apply(json.loads)

In [202]:
fk_schema = {
  "$id": "https://example.com/person.schema.json",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "Foreign key",
  "type": "array",
  "items": {
      "type": "object"
  }
}

In [203]:
assert all(covidom_struc['json_fk'].apply(lambda fk: jsonschema.validate(fk, fk_schema) is None))

In [204]:
import re

def transf_json_obj(obj, tbl_dest_name):
    col_dest_name = list(obj.keys())[0]
    col_src_name = list(obj[col_dest_name].keys())[0]
    tbl_src_name = obj[col_dest_name][col_src_name]
    
    fk = row.json_fk[0]
    dest_field= list(fk.keys())[0]
    src_field=list(fk[dest_field].keys())[0]
    src_tbl=fk[dest_field][src_field]
    
    return {'tbl_dest_name': tbl_dest_name, 'col_dest_name': col_dest_name, 
            'tbl_src_name': tbl_src_name, 'col_src_name': col_src_name}

def transf_json_objs(arr, tbl_dest_name):
    return list(map(lambda obj: transf_json_obj(obj, tbl_dest_name), arr))
    

In [205]:
assert transf_json_obj(json.loads('{"#PatientUId": {"patient": "patient"}}'), "xx") ==\
{"tbl_dest_name": "xx", "col_dest_name": "#PatientUId", "tbl_src_name": "patient", "col_src_name": "patient"}
assert transf_json_obj(json.loads('{"x": {"y": "z"}}'), "00") ==\
{"tbl_dest_name": "00", "col_dest_name": "x", "tbl_src_name": "z", "col_src_name": "y"}

assert transf_json_objs(json.loads('[{"x": {"y": "z"}}]'), "00") ==\
[{"tbl_dest_name": "00", "col_dest_name": "x", "tbl_src_name": "z", "col_src_name": "y"}]

assert transf_json_objs(covidom_struc.json_fk[3], covidom_struc.table[3]) ==\
[{'tbl_dest_name': 'clinic_followup_reason',
  'col_dest_name': '#Clinic',
  'tbl_src_name': 'clinic',
  'col_src_name': 'Id'}]

#'ALTER TABLE clinic_followup_reason\n  ADD FOREIGN KEY (#Clinic) REFERENCES clinic(Id);\n'

In [206]:
covidom_struc['json_formatted'] = covidom_struc.apply(lambda row: transf_json_objs(row.json_fk, row.table), axis=1)

In [207]:
assert transf_json_objs(covidom_struc.json_fk[3], covidom_struc.table[3]) == [{'tbl_dest_name': 'clinic_followup_reason',
  'col_dest_name': '#Clinic',
  'tbl_src_name': 'clinic',
  'col_src_name': 'Id'}]

In [274]:
list_fks_in_tbl = transf_json_objs(covidom_struc.json_fk[5], covidom_struc.table[5])
list_fks_in_tbl

[{'tbl_dest_name': 'patient_stay',
  'col_dest_name': 'PatientStay_PatientUId',
  'tbl_src_name': 'patient',
  'col_src_name': '#Patient'},
 {'tbl_dest_name': 'patient_stay',
  'col_dest_name': 'PatientStay_ClinicId',
  'tbl_src_name': 'clinic_followup_reason',
  'col_src_name': '#Clinic'},
 {'tbl_dest_name': 'patient_stay',
  'col_dest_name': 'PatientStay_DoctorId',
  'tbl_src_name': 'doctor_list',
  'col_src_name': '#Doctor'},
 {'tbl_dest_name': 'patient_stay',
  'col_dest_name': 'PatientStay_FollowUpReasonId',
  'tbl_src_name': 'clinic_followup_reason',
  'col_src_name': '#Follow up reason'},
 {'tbl_dest_name': 'patient_stay',
  'col_dest_name': 'PatientStay_EndReasonId',
  'tbl_src_name': 'end_reason',
  'col_src_name': 'Id'},
 {'tbl_dest_name': 'patient_stay',
  'col_dest_name': 'PatientStay_MedicalDepartmentId',
  'tbl_src_name': 'medical_department_list',
  'col_src_name': '#Medical dept'}]

In [370]:
def grp_by_src_name(list_fks_in_tbl):
    return reduce(lambda acc, cur: {**acc, cur['tbl_src_name']: [*acc[cur['tbl_src_name']], cur]
                                                   if cur['tbl_src_name'] in acc.keys() else [cur] },list_fks_in_tbl, {})
fks_grpby_src_name = grp_by_src_name(list_fks_in_tbl)
assert fks_grpby_src_name == {'patient': [{'tbl_dest_name': 'patient_stay',
   'col_dest_name': 'PatientStay_PatientUId',
   'tbl_src_name': 'patient',
   'col_src_name': '#Patient'}],
 'clinic_followup_reason': [{'tbl_dest_name': 'patient_stay',
   'col_dest_name': 'PatientStay_ClinicId',
   'tbl_src_name': 'clinic_followup_reason',
   'col_src_name': '#Clinic'},
  {'tbl_dest_name': 'patient_stay',
   'col_dest_name': 'PatientStay_FollowUpReasonId',
   'tbl_src_name': 'clinic_followup_reason',
   'col_src_name': '#Follow up reason'}],
 'doctor_list': [{'tbl_dest_name': 'patient_stay',
   'col_dest_name': 'PatientStay_DoctorId',
   'tbl_src_name': 'doctor_list',
   'col_src_name': '#Doctor'}],
 'end_reason': [{'tbl_dest_name': 'patient_stay',
   'col_dest_name': 'PatientStay_EndReasonId',
   'tbl_src_name': 'end_reason',
   'col_src_name': 'Id'}],
 'medical_department_list': [{'tbl_dest_name': 'patient_stay',
   'col_dest_name': 'PatientStay_MedicalDepartmentId',
   'tbl_src_name': 'medical_department_list',
   'col_src_name': '#Medical dept'}]}

In [305]:
fks_grpby_src_name = covidom_struc['json_formatted'].apply(grp_by_src_name)

In [310]:
def fks_merge_cols(fks_grpby_src_name):
    return {key : reduce(lambda acc, cur: { 'tbl_dest_name': cur['tbl_dest_name'],
                                 'col_dest_name': [cur['col_dest_name'], *acc['col_dest_name']] if 'col_dest_name' in acc else [cur['col_dest_name']],
                                 'tbl_src_name': cur['tbl_src_name'],
                                 'col_src_name': [cur['col_src_name'], *acc['col_src_name']] if 'col_src_name' in acc else [cur['col_src_name']],
}, value, {}) for key, value in fks_grpby_src_name.items()}

In [368]:
fks_merged_cols = fks_grpby_src_name.apply(fks_merge_cols)

In [369]:
assert fks_merged_cols[5]['clinic_followup_reason'] == {'tbl_dest_name': 'patient_stay',
  'col_dest_name': ['PatientStay_FollowUpReasonId', 'PatientStay_ClinicId'],
  'tbl_src_name': 'clinic_followup_reason',
  'col_src_name': ['#Follow up reason', '#Clinic']}

In [349]:
alter_tmplt = Template("""ALTER TABLE $table_dest
  ADD FOREIGN KEY ($dest_field) REFERENCES $source_table($source_field);
""")

In [350]:
def fk_sql_from_tplt_ho(template):
    def list2upplet(inp):
        return ', '.join(map(lambda e: "\"" + e + "\"", inp)) if isinstance(inp, list) else inp
    def foreign_key_sql_from_tplt(jsons):
        return '\n'.join(list(map(
            lambda jso: template.substitute(
                table_dest=jso["tbl_dest_name"],
                dest_field=list2upplet(jso["col_dest_name"]),
                source_field=list2upplet(jso["col_src_name"]),
                source_table=jso["tbl_src_name"]),
            jsons.values())))
    return foreign_key_sql_from_tplt
foreign_key_sql= fk_sql_from_tplt_ho(alter_tmplt)


In [353]:
row = covidom_struc.loc[5]
fk = row.json_fk[5]
dest_field= list(fk.keys())[0]
src_field=list(fk[dest_field].keys())[0]
src_tbl=fk[dest_field][src_field]

alter_tmplt.substitute(table_dest=row.table, dest_field=dest_field, 
                       source_table=src_tbl, source_field=src_field)

'ALTER TABLE patient_stay\n  ADD FOREIGN KEY (PatientStay_MedicalDepartmentId) REFERENCES medical_department_list(#Medical dept);\n'

In [357]:
sql_scripts = fks_merged_cols.apply(foreign_key_sql)

In [364]:
sql_scripts.str.cat()#sep='\n')

'ALTER TABLE clinic_followup_reason\n  ADD FOREIGN KEY ("#Clinic") REFERENCES clinic("Id");\nALTER TABLE patient_note\n  ADD FOREIGN KEY ("PatientUId") REFERENCES patient("#Patient");\nALTER TABLE patient_stay\n  ADD FOREIGN KEY ("PatientStay_PatientUId") REFERENCES patient("#Patient");\n\nALTER TABLE patient_stay\n  ADD FOREIGN KEY ("PatientStay_FollowUpReasonId", "PatientStay_ClinicId") REFERENCES clinic_followup_reason("#Follow up reason", "#Clinic");\n\nALTER TABLE patient_stay\n  ADD FOREIGN KEY ("PatientStay_DoctorId") REFERENCES doctor_list("#Doctor");\n\nALTER TABLE patient_stay\n  ADD FOREIGN KEY ("PatientStay_EndReasonId") REFERENCES end_reason("Id");\n\nALTER TABLE patient_stay\n  ADD FOREIGN KEY ("PatientStay_MedicalDepartmentId") REFERENCES medical_department_list("#Medical dept");\nALTER TABLE doctor\n  ADD FOREIGN KEY ("#Doctor") REFERENCES doctor_list("#Doctor");\n\nALTER TABLE doctor\n  ADD FOREIGN KEY ("#Clinic") REFERENCES clinic("Id");\n\nALTER TABLE doctor\n  ADD F